### Imports and CUDA

In [334]:
# Matplotlib
import requests
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Torch
import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [335]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Objective

#### To develop a model that predicts taxi availability within a specific area for the next three hours. This means that if the model is run at 12 PM, it will provide predicted taxi availability for 1 PM, 2 PM, and 3 PM.

The area of interest is defined by the following geographical boundaries:

    North: 1.35106
    South: 1.32206
    East: 103.97839
    West: 103.92805

To identify the taxis currently available within this region, we use the TaxiAvailabilityScript.py.

This script collects real-time data, which serves as input for our predictive model.

By leveraging historical taxi availability trends and real-time data, our model aims to provide accurate forecasts, helping commuters, ride-hailing services, and urban planners make informed decisions.


# **To-Do List for Taxi Availability Prediction**

## **Step 1: Cleaning the Taxi Availability Data**
The first step involves retrieving and preprocessing the taxi availability dataset. The dataset consists of the following columns:

1. **DateTime**  
2. **Taxi Available Throughout Singapore**  
3. **Taxi Available in Selected Box Area**  
4. **Coordinates[]**  

For our specific use case, **the coordinates column will not be used for now**.  

To prepare the data for the neural network:  
- **Inputs:** We will use `DateTime` and `Taxi Available Throughout Singapore` as features.  
- **Output:** `Taxi Available in Selected Box Area` will be the target variable.  
- **DateTime Conversion:** Since `DateTime` is not in a format suitable for neural networks, we will extract relevant features:
  - **IsWeekend**: A binary feature (1 if it's a weekend, 0 otherwise).  
  - **Hour**: Transformed into a numerical value between **1 and 24** (avoiding 0, which may cause training issues).  

---

## **Step 2: Adding Additional Features**  
*(Partially completed; will be refined over time)*  

Aside from the existing columns, we aim to incorporate additional features that may improve prediction accuracy:  

1. **ERP Rates (Electronic Road Pricing) at the given time and location**  
   - Uncertain if this will significantly impact predictions. Further analysis is needed.  

2. **Number of LTA (Land Transport Authority) gantry locations**  
   - Again, its usefulness remains uncertain—further evaluation required.  

3. **Traffic Incidents in the Selected Area**  
   - A script (`TrafficIncidentScript.py`) has been written to update `traffic_incident.csv` with the latest traffic incidents.  
   - Over time, as the dataset grows, we hope this feature will become useful.  

4. **Number of Taxi Stands in the Area**  
   - Currently **not useful** because our area of interest is fixed.  
   - However, if we allow dynamic selection of areas in the future, this could become relevant.  

5. **Temperature at a Given Time and Date** *(To be implemented)*  

6. **Rainfall Data** *(To be implemented)*  

To ensure all features align properly, we will **synchronize all datasets based on DateTime** before feeding them into the model.  

---

## **Step 3: Creating the Training-Test Split**  
- Initially, we will perform an **80/20 Training-Test split** for simplicity.  
- In the future, we may introduce a **Training-Validation-Test split** to further refine model performance.  

---

## **Step 4: Building the Model**  
We will begin with an **LSTM model**, as LSTMs are well-suited for time-series forecasting.  
- **Initial Limitation:** The model, in its basic form, will only predict the next hour.  
- **Future Improvement:** A **sliding window approach** will be explored and implemented to extend predictions further.  

---

## **Step 5: Model Evaluation and Improvement**  
- After the initial model is trained, we will assess its performance.  
- Based on results, we will explore potential improvements, such as hyperparameter tuning, architectural modifications, or additional feature engineering.  

---

This structured approach will guide the development of a robust and accurate taxi availability prediction model. 🚖💡


## **Preparing the taxi_availability data here.**

Normalization of certain inputs are done as well, but I am unsure if it is the right thing to do as well.

In [336]:
# weather_file_path = "merged_weather.csv"
# weather_df = pd.read_csv(weather_file_path, delimiter=",")

# weather_df = weather_df[::-1]


In [337]:
merged_weather_taxi_df = "merged_file_with_mean.csv"
taxi_df = pd.read_csv(merged_weather_taxi_df, delimiter = ",")

#Adjusting for weather parameters
taxi_df = taxi_df.drop(columns = "stationId")

#Adjusting for taxi_vailability parameters
taxi_df_coordinates = taxi_df["Coordinates[]"]
taxt_df_datetime = taxi_df["DateTime"]
taxi_df = taxi_df.drop(columns = "Coordinates[]")
taxi_df = taxi_df.drop(columns = "Taxi Available in Selected Box Area")
taxi_df["DateTime"] = pd.to_datetime(taxi_df["DateTime"])

taxi_df["IsWeekend"] = (taxi_df["DateTime"].dt.weekday >= 5).astype(int)
taxi_df["Hour"] = taxi_df["DateTime"].dt.hour + 1  # Convert 0-23 to 1-24
taxi_df = taxi_df.drop(columns = "DateTime")

print(taxi_df.dtypes)

Taxi Available throughout SG      int64
temp_value                      float64
humidity_value                  float64
rainfall_value                  float64
peak_period                       int64
Group                             int64
Average Taxi Availability       float64
IsWeekend                         int64
Hour                              int32
dtype: object


### Converting all dtypes into float32

In [338]:
# taxi_df=taxi_df[:5120]
numeric_columns = taxi_df.select_dtypes(include=['int64', 'int32','float64','object']).columns
print("numeric_columns",numeric_columns)
taxi_df[numeric_columns] = taxi_df[numeric_columns].astype('float32')
numeric_columns = taxi_df.select_dtypes(include=['int64', 'int32','float64','object']).columns

# Convert selected columns to float32
taxi_df[numeric_columns] = taxi_df[numeric_columns].astype('float32')

numeric_columns Index(['Taxi Available throughout SG', 'temp_value', 'humidity_value',
       'rainfall_value', 'peak_period', 'Group', 'Average Taxi Availability',
       'IsWeekend', 'Hour'],
      dtype='object')


### Normalizing all values

In [339]:
# #---------------Normalise-----------------------
# Drop 'DateTime' as it's no longer needed
# Normalize the 'Hour' and 'IsWeekend' columns (if needed)

data_min = taxi_df.min(axis=0)
data_max = taxi_df.max(axis=0)
taxi_df_normalized = (taxi_df - data_min) / (data_max - data_min)

# scaler = MinMaxScaler()
# taxi_df[["Hour", "IsWeekend", "Taxi Available throughout SG","Taxi Available in Selected Box Area"]] = scaler.fit_transform(taxi_df[["Hour", "IsWeekend","Taxi Available throughout SG","Taxi Available in Selected Box Area"]])
# taxi_df = taxi_df.apply(pd.to_numeric, errors='coerce')
taxi_df_output_normalized  = taxi_df_normalized["Average Taxi Availability"]
taxi_df_output_training_normalized = taxi_df_normalized["Average Taxi Availability"]
taxi_df_normalized = taxi_df_normalized.drop(columns = "Average Taxi Availability")
taxi_df_normalized = taxi_df_normalized.drop(columns = "Group")

taxi_df_normalized.to_csv("checker.csv", index=False)  # Set index=False to exclude row numbers

# Convert to NumPy arrays
input_data = taxi_df_normalized.values  # Shape: (5120, num_features)
output_data = taxi_df_output_normalized.values  # Shape: (5120,)
output_training_data = taxi_df_output_training_normalized.values
# taxi_df_output = taxi_df["Taxi Available in Selected Box Area"]
# input_data = taxi_df.values  # Shape: (5120, num_features)
# output_data = taxi_df_output.values  # Shape: (5120,)

print("Input Data: ",input_data.shape)
print("Output Data: ",output_data.shape)
print("Training Output Data: ", output_training_data.shape)

# Define sequence length

Input Data:  (25591, 7)
Output Data:  (25591,)
Training Output Data:  (25591,)


### No Normalization Style

In [340]:
# #---------------No Normalization-----------------------

# # Drop 'DateTime' as it's no longer needed
# taxi_df_no_norm = taxi_df  # Remove DateTime but keep raw values

# # Separate input and output data
# taxi_df_output_no_norm = taxi_df_no_norm["Taxi Available in Selected Box Area"]
# taxi_df_no_norm = taxi_df_no_norm.drop(columns=["Taxi Available in Selected Box Area"])

# # Save to CSV for checking
# taxi_df_no_norm.to_csv("checker_no_norm.csv", index=False)  # Set index=False to exclude row numbers

# # Convert to NumPy arrays (raw values)
# input_data = taxi_df_no_norm.values  # Shape: (5120, num_features)
# output_data = taxi_df_output_no_norm.values  # Shape: (5120,)

# print("Input Data: ", input_data.shape)
# print("Input Data: ", input_data[0])

# print("Output Data: ", output_data.shape)
# print("Input Data: ", output_data[0])


### Create Sequence Function

In [341]:
seq_length = 24
pred_horizon = 3  # Number of future time steps to predict

def create_sequences(data, labels, seq_length, pred_horizon):
    xs, ys = [], []
    for i in range(0, len(data), seq_length):  # Start from 0 and increment by seq_length
        if i + seq_length + pred_horizon <= len(data):  # Ensure enough data for prediction horizon
            xs.append(data[i:i + seq_length])  # Input sequence (continuous)
            ys.append(labels[i + seq_length : i + seq_length + pred_horizon])  # Next 3 values
    return np.array(xs), np.array(ys)

In [342]:

# X, y = create_sequences(input_data, output_data, seq_length,pred_horizon)
X, y = create_sequences(input_data, output_training_data, seq_length,pred_horizon)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y[:, None], dtype=torch.float32)
y = y.permute(0, 2, 1)  # Shape: (samples, pred_horizon, 1)

# Split sizes
total_samples = len(X)
train_size = int(0.8 * total_samples)
val_size = int(0.1 * total_samples)
test_size = total_samples - train_size - val_size

# Split the data
trainX, valX, testX = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
trainY, valY, testY = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

# Create TensorDatasets
train_dataset = TensorDataset(trainX, trainY)
val_dataset = TensorDataset(valX, valY)
test_dataset = TensorDataset(testX, testY)

# DataLoaders
batch_size = 17
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

# Example of accessing a batch of data
for inputs, targets in train_loader:
    print(f'Inputs: {inputs.shape}, Targets: {targets.shape}')
    break  # Only print the first batch for verification

print("X[1]: ", X[1][1])
print("Y[1]: ",y[1][1])

Inputs: torch.Size([17, 24, 7]), Targets: torch.Size([17, 3, 1])
X[1]:  tensor([0.5725, 0.8018, 0.8426, 0.0000, 1.0000, 0.0000, 0.9565])
Y[1]:  tensor([0.4312])


In [343]:
print("test_loader",len(test_loader))

test_loader 6


In [344]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, output_size, num_heads, num_layers, hidden_dim, dropout=0.1):
        super(TransformerModel, self).__init__()
        
        self.embedding = nn.Linear(input_size, hidden_dim)         # input_size = 3
        self.decoder_embedding = nn.Linear(1, hidden_dim)        
        self.encoder_positional_encoding = nn.Parameter(torch.rand(1, 100, hidden_dim)) 
        self.decoder_positional_encoding = nn.Parameter(torch.rand(1, 100, hidden_dim))  # max_seq_len=100


        self.transformer = nn.Transformer(
            d_model=hidden_dim, 
            nhead=num_heads, 
            num_encoder_layers=num_layers, 
            num_decoder_layers=num_layers, 
            dim_feedforward=hidden_dim * 2, 
            dropout=dropout,
            batch_first=True
        )

        self.fc_out = nn.Linear(hidden_dim, output_size)
    
    def forward(self, src, tgt=None):
        src = self.embedding(src)
        src += self.encoder_positional_encoding[:, :src.size(1), :]

        
        if tgt is not None:
            tgt_input = self.decoder_embedding(tgt)
            tgt_input += self.decoder_positional_encoding[:, :tgt_input.size(1), :]
        else:
            print("target is using source")
            tgt_input = src

        output = self.transformer(src, tgt_input)
        output = self.fc_out(output)
        return output


In [345]:
def train_transformer(model, dataloader, num_epochs, learning_rate, device):
    # Set the loss function and optimizer
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()  # Set the model to training mode
    loss_graph = []
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            
        

            # Forward pass
            output = model(inputs, tgt=targets)

            # Match shapes before computing loss
            
            print("output:", output.shape)
            print("target:", targets.shape)

            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            epoch_loss += loss.item()
        
        loss_graph.append(epoch_loss / len(dataloader))
        
        # Print the loss every 50 epochs
        if epoch % 50 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(dataloader)}")
    
    print(f"Final Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(dataloader)}")
    return loss_graph

In [346]:
# Define the model parameters
# Following the research paper's instructions
input_size = 7
hidden_size = 64
num_layers = 1
output_size = 1
num_heads = 8
dataloader = train_loader
#Create the model
model = TransformerModel(input_size, output_size, num_heads, num_layers, hidden_dim = hidden_size)
loss_graph = train_transformer(model, dataloader, num_epochs=300, learning_rate=0.01, device=device)
plt.plot(loss_graph)
plt.title("Loss Graph")
plt.xlabel("Epochs")
plt.ylabel("Loss")

output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: torch.Size([17, 3, 1])
target: torch.Size([17, 3, 1])
output: 

KeyboardInterrupt: 

In [ ]:
print(len(test_loader))

4


In [ ]:
model.eval()  # Set the model to evaluation mode

# Initialize variables to track loss
loss_value = 0
num_batches = 0

# Define the loss function
criterion = torch.nn.MSELoss()

# Disable gradient computation for validation
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        if batch_idx == len(test_loader) - 1:
            break  # Skip the last batch if needed (e.g., incomplete sequence)

        # Move data to device
        inputs, targets = inputs.to(device), targets.to(device)  # inputs: [B, 24, 7], targets: [B, 3, 1]

        # Initialize decoder input with a fixed token, e.g., 0.0
        batch_size = inputs.size(0)
        tgt_input = torch.zeros(batch_size, 1, 1).to(device)  # shape: [B, 1, 1]

        # Autoregressive prediction
        for step in range(targets.size(1)):  # Predict 3 steps
            output = model(inputs, tgt=tgt_input)  # output: [B, current_step+1, 1]
            next_token = output[:, -1:, :]  # [B, 1, 1]
            tgt_input = torch.cat([tgt_input, next_token], dim=1)  # Grow decoder input

        # Skip the first token (initial start token)
        predicted = tgt_input[:, 1:, :]  # [B, 3, 1]

        # Denormalize predictions and targets
        output_denorm = predicted * (data_max["Taxi Available in Selected Box Area"] - data_min["Taxi Available in Selected Box Area"]) + data_min["Taxi Available in Selected Box Area"]
        targets_denorm = targets * (data_max["Taxi Available in Selected Box Area"] - data_min["Taxi Available in Selected Box Area"]) + data_min["Taxi Available in Selected Box Area"]

        # Compute loss on normalized data
        loss_value += criterion(predicted, targets)

        # Print a sample of the normalized and denormalized values
        print("Normalized predicted[0]:", predicted[0].squeeze().tolist())
        print("Normalized target[0]:   ", targets[0].squeeze().tolist())
        print("Denormalized predicted[0]:", output_denorm[0].squeeze().tolist())
        print("Denormalized target[0]:   ", targets_denorm[0].squeeze().tolist())
        print("-" * 50)

        num_batches += 1

# Compute average loss
loss_value = loss_value / num_batches
print("Predicted output shape:", predicted.shape)
print("True output shape:", targets.shape)
print(f'Average Validation Loss: {loss_value:.4f}')


Normalized output[0]: [[0.42491087317466736, 0.4275411367416382, 0.4214435815811157], [0.42491087317466736, 0.4275411367416382, 0.4214435815811157], [0.42491087317466736, 0.4275411367416382, 0.4214435815811157], [0.42491087317466736, 0.4275411367416382, 0.4214435815811157], [0.42491087317466736, 0.4275411367416382, 0.4214435815811157], [0.42491087317466736, 0.4275411367416382, 0.4214435815811157], [0.42491090297698975, 0.4275411069393158, 0.4214435815811157], [0.42491084337234497, 0.4275411367416382, 0.4214435815811157], [0.42491087317466736, 0.4275411367416382, 0.4214435815811157], [0.42491090297698975, 0.4275411069393158, 0.4214435815811157], [0.42491087317466736, 0.4275411367416382, 0.4214435815811157], [0.42491090297698975, 0.4275411069393158, 0.4214435815811157], [0.42491084337234497, 0.4275411367416382, 0.4214435815811157], [0.42491084337234497, 0.4275411367416382, 0.4214435815811157], [0.42491084337234497, 0.4275411367416382, 0.4214435815811157], [0.42491087317466736, 0.42754113

In [ ]:

# model.eval()

# # Initialize variables to track loss
# loss_value = 0
# num_batches = 0

# # Define the loss function
# criterion = torch.nn.MSELoss()

# # Initialize hidden state and cell state
# hidden_state, cell_state = None, None  

# # Disable gradient computation for validation
# with torch.no_grad():
#     for batch_idx, (inputs, targets) in enumerate(test_loader):
#         if batch_idx == len(test_loader) - 1:  
#             break  # Skip the last batch
#         # Forward pass
#         output, cell_state, hidden_state = model(inputs, cell_state, hidden_state)
#         print("Predicted output: ", output)
#         print("True Output: ", targets)

#         # Compute loss
#         # Compute loss
#         loss_value += criterion(output, targets)
# # Compute average loss
# loss_value = loss_value / (len(test_loader) -1 )

# # Print validation results
# print(f'Average Validation Loss: {loss_value:.4f}')
